# Feature Vector Calculations
The purpose of this notebook is to write out formulas for calculating feature vector updates and to document the thought process for the implementation of feature vectors and the dictionary outside of the final program.

## Statistical Calculations

### Average
Below is the formula for calculating the current average, $\mu_{n}$, for different fields with only the calculated average from one previous time step, $\mu_{n-1}$, and the current value update for the field, $x_{n}$. This can be used on any of the fields present in the track dataset.

$$
\mu_{n} = \frac{(n-1)\mu_{n-1} + x_{n}}{n}
$$

### Standard Deviation
Below is the formula for calculating the current standard deviation, $\sigma_{n}$, with only the previously calculated standard deviation from the last time step, $\sigma_{n-1}$, and the current value update for the field, $x_{n}$. This can be used on any of the fields present in the track dataset.

$$
\sigma_{n} = \sqrt{\frac{(n-1)\sigma_{n-1}^{2} + (x_{n}-\mu_{n})^2}{n}}
$$

## Smoothness Calculations

Below are the formulas for calculating both the smoothness $M(1)$ and $M(2)$ metrics. These metrics convey the relative visual smoothness of a signal processed from any of the fields either provided by the kinematic track data or features calculated throughout the script. Added variables $q$ and $c$ are also specified in the equations. $z_n$ is any field or calculated feature at the current time interval.
$$
q_{n} = \frac{1}{n}(q_{n-1} * (n-1) + (z_{n} - z_{n-1}))
$$
$$
M(1)_n = \frac{1}{n}(M(1)_{n-1}*(n-1)+(z_{n}-z_{n-1}-q)^2)
$$
$$
c_n = \frac{1}{n}(c_{n-1}*(n-1)+|z_{n}-z_{n-1}|)
$$
$$
M(2)_n=\frac{M(1)_n}{c_n}
$$

## Euclidean Distance - 3D

Below is the three dimensional euclidean distance formula. For constructing the three dimensional coordinate space, we will use $latitude$ and $longitude$ of each track update as the $x$ and $y$ axis and $altitude$ as the $z$ axis. 

$$ d_{n}=||p-q|| = \sqrt{(lat_{a}-lat_{b})^2+(lon_{a}-lon_{b})^2+(alt_{a}-alt_{b})^2}$$

## Curvature

Using the three dimensional euclidean distance formula show above, curvature is calculated using the formula shown below where $a$, $b$, and $c$ are the calculated euclidean distances for 3 consecutive track updates. This curvature formula will be considered a short term measurement of curvature, work is being done to investigate what a longer interval curvature measurement would look like.

$$ 
\kappa_{n} = cos^{-1}\frac{a^2-b^2-c^2}{2bc}
$$

## Heading

$$
h_{n} = tan^{-1}(lon_{n-1} - lon_{n}, lat_{n-1} - lat_{n})
$$

## Manuverability Score

$$
\phi_{n} = \frac{\mu_{speed}}{\sigma_{heading}}
$$

### Current Feature List

1. Average of speed
2. Average of heading
3. Average of curvature
4. Standard Deviation of speed
5. Standard Deviation of heading
6. Manuverability Score
7. M(1) Smoothness of range
8. M(1) Smoothness of azimuth
9. M(1) Smoothness of elevation
10. M(1) Smoothness of speed
11. M(1) Smoothness of heading
12. M(2) Smoothness of range
13. M(2) Smoothness of azimuth
14. M(2) Smoothness of elevation
15. M(2) Smoothness of speed
16. M(2) Smoothness of heading